In [0]:
import numpy as np
from keras import models
from keras import layers
from keras import optimizers
import matplotlib.pyplot as plt
  
  
# ******************************************************
#                Upload data
# ******************************************************
from google.colab import files
uploaded = files.upload()
print(uploaded.keys())

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
bj = np.load('BJ.npz')
BJ = bj['BJ']


# Design parameters
tabNum=3
trainNum = 250


# Normalize
mean = BJ.mean(axis=0)
std = BJ.std(axis=0)
data = (BJ-mean)/std


# data processing
u = data[:, 0]
y = data[:, 1]
dataNum, _ = data.shape


X = np.zeros((dataNum - tabNum, 2*tabNum))
#X = [y(k-3:k-1,1); u(k-2:k,1)]
#X = [y(k-3) y(k-2) y(k-1) u(k-2) u(k-1) u(k)]


for k in range(tabNum, dataNum):
    x = np.array(np.concatenate((y[k-tabNum:k], u[k-tabNum+1:k + 1]), axis=None), ndmin=2)
    X[k-tabNum, :] = x

Y = np.array(y[tabNum:dataNum], ndmin=2)  # Row wise data set
Y = Y.T

Ttrain = np.array(range(tabNum, trainNum)) - tabNum
Ttest = np.array(range(trainNum, dataNum)) - tabNum
x_train = X[Ttrain, :]
y_train = Y[Ttrain, :]
x_test = X[Ttest, :]
y_test = Y[Ttest, :]

In [0]:
print(x_train)

In [0]:
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import models
from tensorflow.keras import layers
import os


_, inputdim = x_train.shape
_, outputdim = y_train.shape

# Training Parameters
epochs = 200
batch_size = 32

hiddennum1 = 32
hiddennum2 = 32


# *************************************************************
#               Model building
# *************************************************************

model = models.Sequential()
model.add(layers.Dense(hiddennum1, activation='sigmoid', input_shape=(inputdim,)))
    # ***********************************
    #
    #     Fill in the blank
    #
    # ***********************************
model.add(layers.Dense(outputdim, activation='linear'))
model.summary()

model.compile(loss='mean_squared_error',
              optimizer=RMSprop(),
              metrics=['mae'])

history = model.fit(x=x_train, y=y_train, epochs=epochs, batch_size=batch_size,
                    validation_data=(x_test, y_test),verbose=2)

history_dict = history.history
history_dict.keys()

# *************************************************************
#               Visualization
# *************************************************************

# Training loss
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()



# Training accuracy
plt.clf()   # 그래프를 초기화합니다
mae = history.history['mean_absolute_error']
val_mae = history.history['val_mean_absolute_error']
plt.plot(epochs, mae, 'bo', label='Training MAE')
plt.plot(epochs, val_mae, 'b', label='Validation MAE')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


In [0]:
#  Load data
timenum = len(u)
datanumnum,inputdim = x_train.shape
tabnum = int(inputdim/2)
_, outputdim = y_train.shape

# Apply MLP to training and test set
yhat_train = model.predict(x_train)
yhat_test = model.predict(x_test)
yhat = np.row_stack((yhat_train,yhat_test))


# Back to original space
Y = y*std[1]+mean[1]
Yhat = yhat*std[1]+mean[1]

# ** ** ** ** ** test ** ** ** ** **
plt.plot(range(timenum), Y)
plt.plot(range(tabnum,timenum),Yhat)
plt.show()
